# Querys

In [1]:
import mysql.connector
import time
import pandas as pd
import datetime

In [2]:
db = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  database='project'
)

In [3]:
cursor = db.cursor()

In [4]:
db = mysql.connector.connect(
  host="localhost",
  user="root",
  password="",
  database='project'
)

cursor = db.cursor()# 1. find the most common room type available.
query = '''
    SELECT room_type, COUNT(*) as frequency
    FROM listing
    GROUP BY room_type
    ORDER BY COUNT(*) DESC
'''
start_time = time.time()
cursor.execute(query)
print(cursor.fetchall())
print('time used for this query is ', time.time() - start_time, 's', sep = '')

[('Entire home/apt', 8116), ('Private room', 5313), ('Hotel room', 664), ('Shared room', 523)]
time used for this query is 0.14333224296569824s


In [5]:
# 2. find top 15 property types
query = '''
    SELECT property_type, COUNT(*) as frequency
    FROM listing
    GROUP BY property_type
    ORDER BY COUNT(*) DESC
    LIMIT 15
'''
start_time = time.time()
cursor.execute(query)
print(cursor.fetchall())
print('time used for this query is ', time.time() - start_time, 's', sep = '')

[('Entire rental unit', 3545), ('Entire condo', 3198), ('Private room in rental unit', 1439), ('Private room in condo', 785), ('Room in boutique hotel', 572), ('Private room in home', 551), ('Room in hotel', 539), ('Entire home', 397), ('Entire serviced apartment', 384), ('Private room in townhouse', 357), ('Private room in hostel', 324), ('Private room in serviced apartment', 305), ('Shared room in hostel', 285), ('Private room in bed and breakfast', 261), ('Entire townhouse', 256)]
time used for this query is 0.010516881942749023s


In [6]:
# 3. find the top 10 property based on price 
query = '''
    SELECT property_type, average_price, frequency
    FROM 
        (SELECT property_type, AVG(price) as average_price, COUNT(*) as frequency
            FROM listing
            GROUP BY property_type
        ) as new_table
    ORDER BY average_price DESC
    LIMIT 10
'''
start_time = time.time()
cursor.execute(query)
print(cursor.fetchall())
print('time used for this query is ', time.time() - start_time, 's', sep = '')

[('Dome', Decimal('51591.250000'), 8), ('Entire villa', Decimal('14371.478261'), 46), ('Shared room in casa particular', Decimal('14126.000000'), 1), ('Private room in boat', Decimal('12000.000000'), 1), ('Barn', Decimal('8850.000000'), 1), ('Shared room in serviced apartment', Decimal('8527.000000'), 3), ('Farm stay', Decimal('6826.000000'), 7), ('Shared room in hostel', Decimal('6152.333333'), 285), ('Private room in nature lodge', Decimal('6089.200000'), 5), ('Entire home', Decimal('5553.068010'), 397)]
time used for this query is 0.013734579086303711s


In [7]:
# 4. find the bottom 10 property based on price
query = '''
SELECT property_type, average_price, frequency
FROM 
    (SELECT property_type, AVG(price) as average_price, COUNT(*) as frequency
        FROM listing
        GROUP BY property_type
    ) as new_table
ORDER BY average_price
LIMIT 10
'''
start_time = time.time()
cursor.execute(query)
print(cursor.fetchall())
print('time used for this query is ', time.time() - start_time, 's', sep = '')

[('Shared room in cave', Decimal('320.000000'), 1), ('Shared room in cabin', Decimal('333.000000'), 1), ('Shared room in chalet', Decimal('375.000000'), 1), ('Private room in earthen home', Decimal('500.000000'), 1), ('Shared room in bed and breakfast', Decimal('507.721311'), 61), ('Shared room in townhouse', Decimal('538.631579'), 38), ('Private room in barn', Decimal('554.250000'), 4), ('Shared room in boutique hotel', Decimal('559.000000'), 4), ('Entire hostel', Decimal('590.000000'), 1), ('Shared room in tiny home', Decimal('671.500000'), 2)]
time used for this query is 0.01319575309753418s


In [8]:
# 5. find the top 10 property based on review score
query = '''
SELECT property_type, average_score, frequency
FROM 
    (SELECT property_type, AVG(review_scores_rating) as average_score, COUNT(*) as frequency
        FROM listing
        WHERE review_scores_rating IS NOT NULL
        GROUP BY property_type
    ) as new_table
ORDER BY average_score DESC
LIMIT 10
'''
start_time = time.time()
cursor.execute(query)
print(cursor.fetchall())
print('time used for this query is ', time.time() - start_time, 's', sep = '')

[('Casa particular', Decimal('5.000000000'), 1), ('Entire hostel', Decimal('5.000000000'), 1), ('Shared room in guesthouse', Decimal('5.000000000'), 2), ('Farm stay', Decimal('5.000000000'), 1), ('Private room in farm stay', Decimal('5.000000000'), 1), ('Private room in resort', Decimal('5.000000000'), 2), ('Private room in kezhan', Decimal('5.000000000'), 3), ('Private room in vacation home', Decimal('5.000000000'), 1), ('Barn', Decimal('5.000000000'), 1), ('Private room in barn', Decimal('5.000000000'), 1)]
time used for this query is 0.010784626007080078s


In [9]:
# 6. find the bottom 10 property based on review score
query = '''
SELECT property_type, average_score, frequency
FROM 
    (SELECT property_type, AVG(review_scores_rating) as average_score, COUNT(*) as frequency
        FROM listing
        WHERE review_scores_rating IS NOT NULL
        GROUP BY property_type
    ) as new_table
ORDER BY average_score
LIMIT 10
'''
start_time = time.time()
cursor.execute(query)
print(cursor.fetchall())
print('time used for this query is ', time.time() - start_time, 's', sep = '')

[('Entire bed and breakfast', Decimal('3.816666666'), 3), ('Private room in floor', Decimal('4.000000000'), 1), ('Entire home/apt', Decimal('4.037500000'), 8), ('Shared room in cave', Decimal('4.250000000'), 1), ('Shared room in rental unit', Decimal('4.284285714'), 7), ('Private room', Decimal('4.340000000'), 23), ('Shared room in bed and breakfast', Decimal('4.366666666'), 12), ('Private room in serviced apartment', Decimal('4.460396039'), 101), ('Entire place', Decimal('4.465000000'), 14), ('Room in aparthotel', Decimal('4.484400000'), 75)]
time used for this query is 0.009935379028320312s


In [10]:
# 1. Find the top host based on the total earnings of all his listings
query = '''
SELECT host_id, host_name, SUM(total_earning) as total_earning, COUNT(*) as number_of_listing, AVG(price) as average_price
FROM
    (SELECT listing.host_id, host.host_name, price, number_of_reviews, minimum_nights, 
            price * number_of_reviews * minimum_nights as total_earning
    FROM listing
    LEFT JOIN host
    ON listing.host_id = host.host_id
    WHERE host_name IS NOT NULL
    ) as new_table
GROUP BY host_id, host_name
ORDER BY SUM(total_earning) DESC
LIMIT 10
'''
start_time = time.time()
cursor.execute(query)
print(cursor.fetchall())
print('time used for this query is ', time.time() - start_time, 's', sep = '')

[(5735895, 'Pariya', Decimal('963339100.00'), 3, Decimal('2500.000000')), (2129668, 'Wimonpak', Decimal('289880984.00'), 40, Decimal('914.425000')), (12464021, 'Rebecca', Decimal('95851520.00'), 21, Decimal('1321.190476')), (12200313, 'Immortal', Decimal('60607920.00'), 4, Decimal('1237.000000')), (19266319, 'Alex', Decimal('59097771.00'), 33, Decimal('1257.666667')), (214380438, 'Lucy', Decimal('49533210.00'), 13, Decimal('1480.846154')), (219301393, 'Jack', Decimal('37237410.00'), 9, Decimal('1930.000000')), (133849226, 'Aidan', Decimal('34845340.00'), 19, Decimal('1942.421053')), (118743647, 'Kate', Decimal('32868000.00'), 4, Decimal('987.500000')), (50093904, 'Leng', Decimal('31025400.00'), 1, Decimal('1050.000000'))]
time used for this query is 0.10062122344970703s


In [11]:
# 2. Find the number of host joined monthly 
query = '''
SELECT MONTH(host_since) as month, YEAR(host_since) as year,  COUNT(*) as number_of_host_joined
FROM host
WHERE host_since IS NOT NULL
GROUP BY month, year
LIMIT 10
        
'''
start_time = time.time()
cursor.execute(query)
print(cursor.fetchall())
print('time used for this query is ', time.time() - start_time, 's', sep = '')

[(12, 2009, 1), (1, 2010, 2), (4, 2010, 1), (5, 2010, 5), (6, 2010, 4), (8, 2010, 2), (10, 2010, 1), (11, 2010, 5), (12, 2010, 1), (1, 2011, 1)]
time used for this query is 0.002920866012573242s


In [12]:
# 3. find the most commonly verified host information
query = '''
SELECT host_id, host_name, host_verifications, host_identity_verified
FROM host
WHERE host_identity_verified IS NOT NULL
LIMIT 10
'''
start_time = time.time()
cursor.execute(query)
print(cursor.fetchall())
print('time used for this query is ', time.time() - start_time, 's', sep = '')

[(58920, 'Gael', "['email', 'phone']", 1), (70413, 'Patiyuth', "['email', 'phone']", 0), (73905, 'Amornpan', "['email', 'phone']", 0), (108793, 'Eddie', "['email', 'phone']", 1), (120437, 'Nuttee', "['email', 'phone']", 1), (120541, 'Emy', "['email', 'phone']", 0), (123784, 'Familyroom', "['email', 'phone']", 0), (124102, 'Giacomo & Rolly', "['email', 'phone']", 1), (132322, 'Dasiri Property', "['email', 'phone']", 1), (152863, 'Nuj', "['email', 'phone']", 0)]
time used for this query is 0.0018055438995361328s


In [13]:
# 4. find the top 10 host with most listings
query = '''
SELECT listing.host_id, host.host_name, COUNT(listing.listing_id) as number_of_listings
FROM listing LEFT JOIN host
ON listing.host_id = host.host_id
GROUP BY listing.host_id, host.host_name
ORDER BY number_of_listings DESC
LIMIT 10
'''
start_time = time.time()
cursor.execute(query)
print(cursor.fetchall())
print('time used for this query is ', time.time() - start_time, 's', sep = '')

[(201677068, 'Curry', 195), (39669202, 'Rfh', 102), (52161947, 'Noons', 81), (26736483, 'K', 71), (187786849, 'Gravity', 65), (228535635, 'Pornchai', 62), (68154488, 'Dusadee', 62), (279089737, 'Oyo', 56), (8444643, 'Kiattisak', 55), (151331046, '卡乐乐', 52)]
time used for this query is 0.01588749885559082s


In [14]:
# 1. find the top 10 listings with lowest price
query = '''
SELECT listing_id,name,property_type,price
FROM listing
ORDER BY price DESC
LIMIT 10
'''
start_time = time.time()
cursor.execute(query)
print(cursor.fetchall())
print('time used for this query is ', time.time() - start_time, 's', sep = '')

[(28374239, 'Onnut BTS Sky Train', 'Shared room in hostel', Decimal('1500000.00')), (41578696, '超奢华私家庄园泳池別墅配豪車保姆', 'Entire villa', Decimal('303309.00')), (13033135, 'Airport-MRT-ShowShopFood-FullFN', 'Private room in home', Decimal('300000.00')), (12056763, 'Charoenkrung Road House/Office Rent', 'Entire home', Decimal('300000.00')), (29970931, '公寓', 'Entire rental unit', Decimal('263896.00')), (32417909, 'asok2', 'Entire condo', Decimal('100000.00')), (4450156, 'Stylish Apartment in Bangkok, MRT', 'Entire rental unit', Decimal('100000.00')), (37841724, 'Suvarnabhumi Pool Villa', 'Dome', Decimal('99999.00')), (37755073, 'พูลวิลล่า สุวรรณภูมิ', 'Dome', Decimal('99999.00')), (37841704, 'Airport Pool Villa', 'Dome', Decimal('99999.00'))]
time used for this query is 0.00928187370300293s


In [15]:
# 2. find the top 10 listings with highest number of reviews
query = '''
SELECT listing_id,name,property_type,number_of_reviews
FROM listing
ORDER BY number_of_reviews DESC
LIMIT 10
'''
start_time = time.time()
cursor.execute(query)
print(cursor.fetchall())
print('time used for this query is ', time.time() - start_time, 's', sep = '')

[(20869092, 'Beautiful One Bedroom Apartment Near skytrain', 'Entire serviced apartment', 1119), (31077175, 'FREE PICK UP⭐BKK AIRPORT/BREAKFAST/PRIVATE DELUXE', 'Private room in hostel', 673), (9453790, "37 Nature House: 10 mins to DMK Int'l Airport", 'Entire cabin', 521), (5964843, 'The Mustang Nero/No.4 The Horse', 'Room in hotel', 480), (6013487, 'The Mustang Nero/No.3 The Wolf', 'Room in hotel', 471), (1026451, '♡Chic Studio, Easy Walk to Pier & BTS Taksin♡', 'Entire rental unit', 449), (11753619, 'near DMK: Don Mueang International Airport', 'Private room in townhouse', 448), (6033632, 'The Mustang Nero/No.2 Black Bird', 'Room in hotel', 426), (393066, '✺✺99 feet in the sky✺✺', 'Entire rental unit', 424), (605314, '☀ ☼Sky bedroom with a smooth life view☼☀', 'Entire rental unit', 422)]
time used for this query is 0.00855398178100586s


In [16]:
# 3. find the top 10 listings with the highest rating score.
query = '''
SELECT listing_id,name,property_type,review_scores_rating
FROM listing
WHERE review_scores_rating IS NOT NULL
ORDER BY review_scores_rating DESC
LIMIT 10
'''
start_time = time.time()
cursor.execute(query)
print(cursor.fetchall())
print('time used for this query is ', time.time() - start_time, 's', sep = '')

[(884503, 'Luxury, resort-like downtown pad', 'Private room in condo', Decimal('5.00000')), (1665970, '1 Bedroom with a Double Bed- 2 guests', 'Entire rental unit', Decimal('5.00000')), (527553, 'Beautiful Apt @ Ratchayothin BTS, Free Wifi', 'Entire condo', Decimal('5.00000')), (1400351, 'Studio in Beung Kum', 'Private room in rental unit', Decimal('5.00000')), (61000, 'Apt.@Pinklao near shopping mall', 'Entire condo', Decimal('5.00000')), (491533, 'Furnished room near BTS Sai Yud', 'Private room in rental unit', Decimal('5.00000')), (430706, '7. Bangkok big bright Apartment 402', 'Entire rental unit', Decimal('5.00000')), (1665776, '1 Bedroom with Twin Beds- 2 guests', 'Entire rental unit', Decimal('5.00000')), (1642375, 'Deluxe studio 45sqm,TheUpRama3,Kitchen,Kid’sClub', 'Room in hotel', Decimal('5.00000')), (1576253, 'Luxury 2BR Condo @ The River Hi Flr', 'Entire condo', Decimal('5.00000'))]
time used for this query is 0.008774995803833008s


In [17]:
# 4. find the top 10 listings with the highest cleaniness score.
query = '''
SELECT listing_id,name,property_type,review_scores_cleanliness
FROM listing
WHERE review_scores_cleanliness IS NOT NULL
ORDER BY review_scores_cleanliness DESC
LIMIT 10
'''
start_time = time.time()
cursor.execute(query)
print(cursor.fetchall())
print('time used for this query is ', time.time() - start_time, 's', sep = '')

[(491533, 'Furnished room near BTS Sai Yud', 'Private room in rental unit', Decimal('5.00000')), (1400351, 'Studio in Beung Kum', 'Private room in rental unit', Decimal('5.00000')), (527553, 'Beautiful Apt @ Ratchayothin BTS, Free Wifi', 'Entire condo', Decimal('5.00000')), (1420577, 'River view near ICON SIAM 74sqm. Free pick up', 'Entire condo', Decimal('5.00000')), (61000, 'Apt.@Pinklao near shopping mall', 'Entire condo', Decimal('5.00000')), (888304, 'Cozy Apt @ Ratchayothin BTS Great Facility', 'Entire condo', Decimal('5.00000')), (884503, 'Luxury, resort-like downtown pad', 'Private room in condo', Decimal('5.00000')), (1629978, 'iSanook Eco Studio', 'Private room in rental unit', Decimal('5.00000')), (1630813, 'iSanook Suite Deluxe 45 sqm', 'Private room in rental unit', Decimal('5.00000')), (430706, '7. Bangkok big bright Apartment 402', 'Entire rental unit', Decimal('5.00000'))]
time used for this query is 0.008318662643432617s


In [18]:
# 5. find the top 10 listing with the highest location score.
query = '''
SELECT listing_id,name,property_type,review_scores_location
FROM listing
WHERE review_scores_location IS NOT NULL
ORDER BY review_scores_location DESC
LIMIT 10
'''
start_time = time.time()
cursor.execute(query)
print(cursor.fetchall())
print('time used for this query is ', time.time() - start_time, 's', sep = '')

[(1665776, '1 Bedroom with Twin Beds- 2 guests', 'Entire rental unit', Decimal('5.00000')), (491533, 'Furnished room near BTS Sai Yud', 'Private room in rental unit', Decimal('5.00000')), (2014542, 'Calm condo 6-meter away to BTS Line', 'Entire rental unit', Decimal('5.00000')), (1642375, 'Deluxe studio 45sqm,TheUpRama3,Kitchen,Kid’sClub', 'Room in hotel', Decimal('5.00000')), (2960072, '2BR Family friendly B&B in old town BKK', 'Room in boutique hotel', Decimal('5.00000')), (1141969, 'Great location on a charming loft', 'Entire condo', Decimal('5.00000')), (2842313, 'Riverside city5mins BTS can sleep 3', 'Entire rental unit', Decimal('5.00000')), (1665970, '1 Bedroom with a Double Bed- 2 guests', 'Entire rental unit', Decimal('5.00000')), (1933894, 'Newly Renovated! 3Br,Single House/Park/BTS/Airport', 'Entire home', Decimal('5.00000')), (884503, 'Luxury, resort-like downtown pad', 'Private room in condo', Decimal('5.00000'))]
time used for this query is 0.008516550064086914s


In [19]:
# 6. find the top 10 host that has the highest response rate.
query = '''
SELECT host_id,host_name, host_response_rate
FROM host
ORDER BY host_response_rate DESC
LIMIT 10
'''
start_time = time.time()
cursor.execute(query)
print(cursor.fetchall())
print('time used for this query is ', time.time() - start_time, 's', sep = '')

[(158654054, 'Erica', '99%'), (151331046, '卡乐乐', '99%'), (21970931, 'Nui', '99%'), (314945555, 'Nana', '99%'), (47324488, 'Tony', '99%'), (218032956, 'Heart', '98%'), (79722098, 'Nuan', '98%'), (50050657, 'Wipa', '98%'), (207453281, 'Hua', '98%'), (104122286, 'Laksanara', '98%')]
time used for this query is 0.0028607845306396484s


In [85]:
# given a review id and a date range, check whether the host of reviewed listing has other listings that are available within the given date range

def check_availability(reviewid, start_date, end_date):
    # date_range = [i.strftime('%Y-%m-%d') for i in pd.date_range(start=datetime.datetime.strptime(start_date, '%Y-%m-%d'), end=datetime.datetime.strptime(end_date, '%Y-%m-%d')).to_pydatetime().tolist()]
    query = f'''
        WITH 
            listingHost AS(
                SELECT host_id AS hostid
                FROM listing
                WHERE listing_id = (SELECT listing_id FROM review WHERE id={reviewid})
            ),
            
            other_listings AS(
                SELECT listing_id AS listingid
                FROM listing
                WHERE host_id = (SELECT hostid FROM listingHost)
            ),
            
            listing_availability AS(
                SELECT listing_id AS listingid
                    , MIN(available) AS availability
                FROM calendar
                WHERE 1 = 1 
                    AND listing_id IN (SELECT listingid FROM other_listings)
                    AND listing_id <> (SELECT listing_id FROM review WHERE id={reviewid})
                    AND date BETWEEN '{start_date}' AND '{end_date}'
                GROUP BY listing_id
            )
            
        SELECT listingid
        FROM listing_availability
        WHERE availability = 1
    '''
    
    cursor.execute(query)
    return cursor.fetchall()


In [86]:
cursor.execute('select MIN(id), listing_id from review GROUP BY listing_id limit 10')
cursor.fetchall()

[(1094339, 27934),
 (85867988, 35780),
 (200452, 55681),
 (180261, 55686),
 (181226, 61000),
 (25689093, 66046),
 (22920345, 145343),
 (885846, 156583),
 (829590, 159854),
 (902543, 166267)]

In [90]:
start_time = time.time()
res = check_availability(200452, '2022-06-30', '2023-02-01')
print(res)
print('time used for this query is ', time.time() - start_time, 's', sep = '')

[(397449,)]
time used for this query is 0.011170148849487305s
